In [ ]:
import json
import os
import sys
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np

## Setup SPARQL / Python Interface

### Define SPARQL Request Function

In [ ]:
#Credit - Doug Fils

def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

### Set SPARQL Endpoint and base URIs

In [ ]:
BCODMO_SERVE = "https://lod.bco-dmo.org/sparql"  #BCO-DMO SPARQL Endpoint
BCODMO_PREF = "http://lod.bco-dmo.org/id/"       #BCO-DMO URI base

## Basic SPARQL Queries
SPARQL is queried in as a string. Can define variables within SPARQL, then query result will be returned in dataframe format which can be used as python variables. Can also return SPARQL query in other formats (like straight json or lists). 

**NOTE:** The BCO-DMO SPARQL endpoint (like many public SPARQL endpoints) limits results to 10,000.

Helpful SPARQL Cheat Sheet: http://www.iro.umontreal.ca/~lapalme/ift6281/sparql-1_1-cheat-sheet.pdf

### Properties ("predicates" in Subject-Predicate-Object model) in BCO-DMO

Also see https://ocean-data.org/schema/ for ontology

#### List all the properties in the BCO-DMO Knowledge Graph

In [ ]:
propertyQuery = '''
SELECT DISTINCT ?property
WHERE { [] ?property [] }
ORDER BY ?property
'''

In [ ]:
propertySearch = get_sparql_dataframe(BCODMO_SERVE, propertyQuery)

In [ ]:
propertySearch.style.set_properties(**{'width': '600px'})

#### Get more information about the properties

In [ ]:
propertyQuery = '''
SELECT DISTINCT ?dataset_parameter ?masterParam
WHERE { ?dataset_parameter odo:isInstanceOf ?masterParam  .
}
'''

In [ ]:
propertySearch = get_sparql_dataframe(BCODMO_SERVE, propertyQuery)

In [ ]:
propertySearch.style.set_properties(**{'width': '600px'})

#### Find out more about "Monitored Properties"

In [ ]:
monitoredPropertiesQuery = '''
SELECT DISTINCT ?masterParam ?shortDesc
WHERE { 
?masterParam odo:hasParameterShortDescription ?shortDesc .
?masterParam owl:deprecated 0 . #Binay - only show those that are not deprecated
}
ORDER BY ?masterParam'''

In [ ]:
monitoredPropertiesSearch = get_sparql_dataframe(BCODMO_SERVE, monitoredPropertiesQuery)

In [ ]:
monitoredPropertiesSearch.style.set_properties(**{'width': '600px'})

#### DESCRIBE a monitored property

In [ ]:
monitoredPropertiesQuery = '''
DESCRIBE <http://lod.bco-dmo.org/id/parameter/901>
'''

In [ ]:
monitoredPropertiesSearch = get_sparql_dataframe(BCODMO_SERVE, monitoredPropertiesQuery)
#ignore the error

In [ ]:
monitoredPropertiesSearch.style.set_properties(**{'width': '600px'})